In [1]:
# instalar paquetes necesarios (ejecutar en una celda con !)
!pip install yfinance pandas numpy scipy scikit-learn matplotlib seaborn statsmodels arch joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 7.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

# Ruta base en Drive
BASE = Path('/content/drive/MyDrive/ProyectoCartera')
# creamos la estructura de carpetas
(BASE / 'data' / 'raw').mkdir(parents=True, exist_ok=True)
(BASE / 'data' / 'processed').mkdir(parents=True, exist_ok=True)
(BASE / 'outputs').mkdir(parents=True, exist_ok=True)

print("Base folder:", BASE)


Mounted at /content/drive
Base folder: /content/drive/MyDrive/ProyectoCartera


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
from pathlib import Path

# Lista de tickers
tickers = ["SPY","QQQ","IWM","EEM","TLT","IEF","AGG","GLD","AAPL","MSFT","GOOG","AMZN","NVDA"]

# Fechas (start inclusive, end exclusive para yfinance)
start = "2015-01-01"
end = "2025-09-21"   # usa la fecha de ayer (2025-09-21)

RAW_DIR = BASE / 'data' / 'raw'
RAW_DIR.exists(), RAW_DIR


(True, PosixPath('/content/drive/MyDrive/ProyectoCartera/data/raw'))

In [4]:
def download_ticker_to_csv(ticker, start, end, outdir: Path, max_retries=3, sleep_between=0.6):
    """
    Descarga historial diario y guarda CSV en outdir/{ticker}.csv.
    Usa auto_adjust=True para que Close esté ajustado por splits/dividends.
    """
    for attempt in range(1, max_retries+1):
        try:
            # auto_adjust=True aplica splits/dividends; threads=False reduce errores en Colab
            df = yf.download(ticker, start=start, end=end, interval="1d",
                             auto_adjust=True, threads=False, progress=False)
            if df is None or df.empty:
                raise ValueError("Empty DataFrame returned")
            # Guardar CSV
            outpath = outdir / f"{ticker}.csv"
            df.to_csv(outpath)
            print(f"[OK] {ticker}  rows:{len(df)} saved to {outpath.name}")
            return True
        except Exception as e:
            print(f"[WARN] Attempt {attempt} failed for {ticker}: {e}")
            time.sleep(sleep_between * attempt)
    print(f"[ERROR] Failed to download {ticker} after {max_retries} attempts")
    return False

# Prueba rápida (no ejecutar todavía, la utilizamos abajo)


In [5]:
results = {}
for t in tickers:
    ok = download_ticker_to_csv(t, start, end, RAW_DIR, max_retries=3, sleep_between=0.6)
    results[t] = ok

print("Summary:")
for t,ok in results.items():
    print(f"  {t}: {'VAMOS TUUU' if ok else 'FAILED'}")


[OK] SPY  rows:2695 saved to SPY.csv
[OK] QQQ  rows:2695 saved to QQQ.csv
[OK] IWM  rows:2695 saved to IWM.csv
[OK] EEM  rows:2695 saved to EEM.csv
[OK] TLT  rows:2695 saved to TLT.csv
[OK] IEF  rows:2695 saved to IEF.csv
[OK] AGG  rows:2695 saved to AGG.csv
[OK] GLD  rows:2695 saved to GLD.csv
[OK] AAPL  rows:2695 saved to AAPL.csv
[OK] MSFT  rows:2695 saved to MSFT.csv
[OK] GOOG  rows:2695 saved to GOOG.csv
[OK] AMZN  rows:2695 saved to AMZN.csv
[OK] NVDA  rows:2695 saved to NVDA.csv
Summary:
  SPY: VAMOS TUUU
  QQQ: VAMOS TUUU
  IWM: VAMOS TUUU
  EEM: VAMOS TUUU
  TLT: VAMOS TUUU
  IEF: VAMOS TUUU
  AGG: VAMOS TUUU
  GLD: VAMOS TUUU
  AAPL: VAMOS TUUU
  MSFT: VAMOS TUUU
  GOOG: VAMOS TUUU
  AMZN: VAMOS TUUU
  NVDA: VAMOS TUUU


In [6]:
# Leer todos los CSV que se guardaron
csvs = sorted(list(RAW_DIR.glob("*.csv")))
print("Found files:", len(csvs))

price_df = pd.DataFrame()
volume_df = pd.DataFrame()

for f in csvs:
    t = f.stem
    df = pd.read_csv(f, index_col=0, parse_dates=True)
    # Verifica que existan las columnas esperadas
    if 'Close' not in df.columns:
        # si usas auto_adjust=False podrías tener 'Adj Close'
        if 'Adj Close' in df.columns:
            df['Close'] = df['Adj Close']
        else:
            print(f"[WARN] {t} missing Close column, skipping.")
            continue
    if 'Volume' not in df.columns:
        print(f"[WARN] {t} missing Volume column (setting NaN).")
        df['Volume'] = np.nan

    price_df[t] = df['Close']
    volume_df[t] = df['Volume']

# Guardar
price_out = BASE / 'data' / 'processed' / 'prices.csv'
vol_out = BASE / 'data' / 'processed' / 'volumes.csv'
price_df.to_csv(price_out)
volume_df.to_csv(vol_out)
print("Saved prices to:", price_out)
print("Saved volumes to:", vol_out)

# Mostrar un vistazo
display(price_df.head())
display(volume_df.head())


Found files: 17


/tmp/ipython-input-1436358058.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, index_col=0, parse_dates=True)
/tmp/ipython-input-1436358058.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, index_col=0, parse_dates=True)
/tmp/ipython-input-1436358058.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, index_col=0, parse_dates=True)
/tmp/ipython-input-1436358058.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent

Saved prices to: /content/drive/MyDrive/ProyectoCartera/data/processed/prices.csv
Saved volumes to: /content/drive/MyDrive/ProyectoCartera/data/processed/volumes.csv


,AAPL,AGG,AMZN,EEM,EFA,GLD,GOOG,GOOGL,IEF,IWM,LQD,MSFT,NVDA,QQQ,SPY,TLT,VNQ
Price,,,,,,,,,,,,,,,,,
Ticker,AAPL,AGG,AMZN,EEM,EFA,GLD,GOOG,GOOGL,IEF,IWM,LQD,MSFT,NVDA,QQQ,SPY,TLT,VNQ
Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,24.214889526367188,81.70874786376953,15.425999641418457,30.35504913330078,43.7806282043457,114.08000183105469,25.973848342895508,26.27894401550293,83.66015625,102.96983337402344,81.63997650146484,39.7677001953125,0.48301154375076294,94.78443908691406,170.58956909179688,94.39940643310547,53.11162567138672
2015-01-05,23.532718658447266,81.8863525390625,15.10949993133545,29.814817428588867,42.74718475341797,115.80000305175781,25.432405471801758,25.778226852416992,84.170654296875,101.59319305419922,81.97377014160156,39.402000427246094,0.4748533368110657,93.39405822753906,167.50881958007812,95.88223266601562,53.40231704711914
2015-01-06,23.534936904907227,82.09351348876953,14.76449966430664,29.689538955688477,42.26298522949219,117.12000274658203,24.84295654296875,25.14203453063965,84.73604583740234,99.83562469482422,82.30760192871094,38.82366943359375,0.46045657992362976,92.14180755615234,165.93106079101562,97.60981750488281,53.93198013305664


,AAPL,AGG,AMZN,EEM,EFA,GLD,GOOG,GOOGL,IEF,IWM,LQD,MSFT,NVDA,QQQ,SPY,TLT,VNQ
Price,,,,,,,,,,,,,,,,,
Ticker,AAPL,AGG,AMZN,EEM,EFA,GLD,GOOG,GOOGL,IEF,IWM,LQD,MSFT,NVDA,QQQ,SPY,TLT,VNQ
Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,212818400,2090200,55664000,65010000,24674800,7109600,28951268,26480000,2028600,46133100,2523600,27913900,113680000,31314600,121465900,9432000,5570500
2015-01-05,257142000,3446200,55484000,83166600,28468200,8177400,41196796,41182000,1521700,51141900,3218800,39673900,197952000,36521300,169632600,9789500,6073700
2015-01-06,263188400,3688100,70380000,52066100,21711600,11238300,57998800,54456000,1890200,67446000,5313400,36447900,197764000,66205500,209151400,18331300,7577100


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path('/content/drive/MyDrive/ProyectoCartera')
RAW_DIR = BASE / 'data' / 'raw'
PROCESSED_DIR = BASE / 'data' / 'processed'

# Leer y construir price_df y volume_df limpios
price_df = pd.DataFrame()
volume_df = pd.DataFrame()

for f in RAW_DIR.glob("*.csv"):
    t = f.stem

    df = pd.read_csv(f, index_col=0, parse_dates=True)

    # Drop the first two rows which contain non-numeric data (if present and relevant)
    # This part might need adjustment based on actual CSV content
    # For yfinance data, usually the first row is valid. Let's inspect raw data if issues persist.
    # For now, remove this line as it was potentially removing valid data if headers were already clean
    # df = df.iloc[2:]

    # Asegurar que index son fechas válidas
    df.index = pd.to_datetime(df.index, errors="coerce")
    df = df[df.index.notna()] # Drop rows where index could not be parsed as date

    # Convertir a numérico (ignorar strings) - yfinance 'Close' is always a column
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

    # Asegurarse de que solo las filas con datos válidos para 'Close' se agreguen
    df = df.dropna(subset=['Close'])

    price_df[t] = df['Close']
    volume_df[t] = df['Volume']

# Quitar columnas que son todas NaN (si alguna no cargó datos)
price_df = price_df.dropna(axis=1, how='all')
volume_df = volume_df.dropna(axis=1, how='all')

# Ordenar fechas
price_df = price_df.sort_index()
volume_df = volume_df.sort_index()

# --- Guardar DataFrames limpios ---
price_out = PROCESSED_DIR / 'prices.csv'
vol_out = PROCESSED_DIR / 'volumes.csv'
price_df.to_csv(price_out)
volume_df.to_csv(vol_out)
print("Saved cleaned prices to:", price_out)
print("Saved cleaned volumes to:", vol_out)

# --- Calcular y guardar retornos ---
# Retornos simples diarios
returns = price_df.pct_change(fill_method=None)
returns = returns.replace([np.inf, -np.inf], np.nan).dropna(how='all') # Limpieza robusta de retornos

returns_out = PROCESSED_DIR / 'returns_daily.csv'
returns.to_csv(returns_out)
print("Saved cleaned returns to:", returns_out)

print("Dimensiones (price_df, volume_df, returns):", price_df.shape, volume_df.shape, returns.shape)
display(price_df.head())
display(returns.head())

/tmp/ipython-input-2351856979.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, index_col=0, parse_dates=True)
/tmp/ipython-input-2351856979.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index, errors="coerce")
/tmp/ipython-input-2351856979.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, index_col=0, parse_dates=True)
/tmp/ipython-input-2351856979.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consiste

Saved cleaned prices to: /content/drive/MyDrive/ProyectoCartera/data/processed/prices.csv
Saved cleaned volumes to: /content/drive/MyDrive/ProyectoCartera/data/processed/volumes.csv
Saved cleaned returns to: /content/drive/MyDrive/ProyectoCartera/data/processed/returns_daily.csv
Dimensiones (price_df, volume_df, returns): (2695, 17) (2695, 17) (2694, 17)


,EFA,LQD,VNQ,GOOGL,QQQ,SPY,EEM,IWM,TLT,IEF,GLD,AGG,AAPL,MSFT,GOOG,NVDA,AMZN
Price,,,,,,,,,,,,,,,,,
2015-01-02,43.780628,81.639977,53.111626,26.278944,94.784439,170.589569,30.355049,102.969833,94.399406,83.660156,114.080002,81.708748,24.214890,39.767700,25.973848,0.483012,15.4260
2015-01-05,42.747185,81.973770,53.402317,25.778227,93.394058,167.508820,29.814817,101.593193,95.882233,84.170654,115.800003,81.886353,23.532719,39.402000,25.432405,0.474853,15.1095
2015-01-06,42.262985,82.307602,53.931980,25.142035,92.141808,165.931061,29.689539,99.835625,97.609818,84.736046,117.120003,82.093513,23.534937,38.823669,24.842957,0.460457,14.7645
2015-01-07,42.732723,82.416664,54.758827,25.068092,93.329628,167.998779,30.331564,101.065063,97.417015,84.720390,116.430000,82.078705,23.864948,39.316936,24.800390,0.459257,14.9210
2015-01-08,43.310886,82.150940,54.965557,25.155432,95.115929,170.979919,30.848312,102.779350,96.126915,84.374794,115.940002,81.952888,24.781883,40.473557,24.878592,0.476533,15.0230


,EFA,LQD,VNQ,GOOGL,QQQ,SPY,EEM,IWM,TLT,IEF,GLD,AGG,AAPL,MSFT,GOOG,NVDA,AMZN
Price,,,,,,,,,,,,,,,,,
2015-01-05,-0.023605,0.004089,0.005473,-0.019054,-0.014669,-0.018059,-0.017797,-0.013369,0.015708,0.006102,0.015077,0.002174,-0.028172,-0.009196,-0.020846,-0.016890,-0.020517
2015-01-06,-0.011327,0.004072,0.009918,-0.024679,-0.013408,-0.009419,-0.004202,-0.017300,0.018018,0.006717,0.011399,0.002530,0.000094,-0.014678,-0.023177,-0.030318,-0.022833
2015-01-07,0.011115,0.001325,0.015331,-0.002941,0.012891,0.012461,0.021625,0.012315,-0.001975,-0.000185,-0.005891,-0.000180,0.014022,0.012705,-0.001713,-0.002606,0.010600
2015-01-08,0.013530,-0.003224,0.003775,0.003484,0.019140,0.017745,0.017037,0.016962,-0.013243,-0.004079,-0.004209,-0.001533,0.038422,0.029418,0.003153,0.037618,0.006836
2015-01-09,-0.004839,0.002653,0.000470,-0.012211,-0.006583,-0.008014,-0.003300,-0.009603,0.010953,0.004933,0.011385,0.002438,0.001073,-0.008405,-0.012951,0.004028,-0.011749


In [8]:
# Retornos simples diarios
returns = price_df.pct_change(fill_method=None).dropna(how='all')

# Guardar
returns_out = '/content/drive/MyDrive/ProyectoCartera/data/processed/returns_daily.csv'
returns.to_csv(returns_out)
print("Saved returns to:", returns_out)

display(returns.head())


Saved returns to: /content/drive/MyDrive/ProyectoCartera/data/processed/returns_daily.csv


,EFA,LQD,VNQ,GOOGL,QQQ,SPY,EEM,IWM,TLT,IEF,GLD,AGG,AAPL,MSFT,GOOG,NVDA,AMZN
Price,,,,,,,,,,,,,,,,,
2015-01-05,-0.023605,0.004089,0.005473,-0.019054,-0.014669,-0.018059,-0.017797,-0.013369,0.015708,0.006102,0.015077,0.002174,-0.028172,-0.009196,-0.020846,-0.016890,-0.020517
2015-01-06,-0.011327,0.004072,0.009918,-0.024679,-0.013408,-0.009419,-0.004202,-0.017300,0.018018,0.006717,0.011399,0.002530,0.000094,-0.014678,-0.023177,-0.030318,-0.022833
2015-01-07,0.011115,0.001325,0.015331,-0.002941,0.012891,0.012461,0.021625,0.012315,-0.001975,-0.000185,-0.005891,-0.000180,0.014022,0.012705,-0.001713,-0.002606,0.010600
2015-01-08,0.013530,-0.003224,0.003775,0.003484,0.019140,0.017745,0.017037,0.016962,-0.013243,-0.004079,-0.004209,-0.001533,0.038422,0.029418,0.003153,0.037618,0.006836
2015-01-09,-0.004839,0.002653,0.000470,-0.012211,-0.006583,-0.008014,-0.003300,-0.009603,0.010953,0.004933,0.011385,0.002438,0.001073,-0.008405,-0.012951,0.004028,-0.011749


In [9]:
print("Tipos de datos:")
print(price_df.dtypes)

print("\nNull counts:")
print(price_df.isna().sum().sort_values(ascending=False).head())


Tipos de datos:
EFA      float64
LQD      float64
VNQ      float64
GOOGL    float64
QQQ      float64
SPY      float64
EEM      float64
IWM      float64
TLT      float64
IEF      float64
GLD      float64
AGG      float64
AAPL     float64
MSFT     float64
GOOG     float64
NVDA     float64
AMZN     float64
dtype: object

Null counts:
EFA      0
LQD      0
VNQ      0
GOOGL    0
QQQ      0
dtype: int64
